In [1]:
ls -lh data

total 272M
-rw-r--r-- 1 stephen stephen 239M Aug  2  2016 data.json
-rw-rw-r-- 1 stephen stephen  33M Dec 17 08:47 data.zip


In [2]:
!wc data/data.json

        0   8173151 250481668 data/data.json


In [7]:
%%writefile subset_json.py
"""head_json.py - extract a couple records from a huge json file.

Syntax: python head_json.py < infile.json > outfile.json
"""

import sys

start_char = '{'
stop_char = '}'
n_records = 1000
level_nesting = 0

while n_records != 0:
    ch = sys.stdin.read(1)
    sys.stdout.write(ch)
    if ch == start_char:
        level_nesting += 1
    if ch == stop_char:
        level_nesting -= 1
        if level_nesting == 0:
            n_records -= 1
sys.stdout.write(']')

Overwriting subset_json.py


In [8]:
!python subset_json.py < data/data.json > data/subset.json

In [1]:
import pandas as pd


df = pd.read_json('data/data.json')

In [2]:
df.acct_type.value_counts(dropna=False)

premium             12373
fraudster_event       851
fraudster             437
spammer_limited       218
spammer_warn          144
tos_warn               91
spammer_noinvite       84
tos_lock               77
locked                 54
fraudster_att           5
spammer_web             2
spammer                 1
Name: acct_type, dtype: int64

In [3]:
df['acct_type'].head(50)

0     fraudster_event
1             premium
2             premium
3             premium
4             premium
5             premium
6             premium
7             premium
8             premium
9             premium
10            premium
11            premium
12            premium
13       spammer_warn
14            premium
15            premium
16            premium
17            premium
18            premium
19            premium
20            premium
21            premium
22            premium
23            premium
24            premium
25            premium
26          fraudster
27            premium
28            premium
29            premium
30            premium
31            premium
32       spammer_warn
33            premium
34            premium
35            premium
36            premium
37            premium
38            premium
39            premium
40            premium
41            premium
42            premium
43            premium
44            premium
45        

In [4]:
df["acct_type"] = df['acct_type'].replace('fraudster_event', 'fraudster')
df["acct_type"] = df['acct_type'].replace('fraudster_att', 'fraudster')

In [5]:
import numpy as np

df['fraud'] = np.where(df['acct_type'] == 'fraudster', True, False)

In [27]:
df[df['fraud'] == True]

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state,fraud
0,fraudster,2010-02-13 12:00:00,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,...,36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL,True
26,fraudster,2011-03-08 18:00:00,870,11,US,USD,0.0,"<h1 class=""post"">Welcome</h1>\r\n<p> </p>\r\n<...",hotmail.com,1299171416,...,42,1295576777,3,400 Fifth Avenue,US,40.750228,-73.983714,The Setai,NY,True
51,fraudster,2012-02-17 08:00:00,536,6,US,USD,0.0,"<p><span style=""font-size: x-large;"">Okay ever...",cox.net,1317544696,...,77,1310957994,1,404 euclid ave,US,32.702072,-117.085088,The Jacobs Center at Market Creek,CA,True
54,fraudster,2011-08-08 00:00:00,3081,11,,GBP,0.0,"<h3><strong><strong><span style=""font-family: ...",hotmail.co.uk,1311717051,...,0,1311717050,1,Hammersmith Road,,0.000000,0.000000,Olympia Exhibition Hall,,True
70,fraudster,2011-12-03 07:00:00,0,13,US,USD,0.0,,yahoo.com,1316795942,...,0,1316795486,1,5801 Security Blvd,US,39.308752,-76.716174,,MD,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14316,fraudster,2013-02-08 08:00:00,3276,4,GB,GBP,0.0,"<div id=""yui_3_7_2_16_1358948795407_159"" style...",hotmail.co.uk,1358948648,...,0,1358948648,1,Sagar Street,,53.523896,-2.276553,Al Maidah Banquet Hall and Restaurant,,True
14318,fraudster,2013-02-02 06:00:00,448,0,US,USD,0.0,<p>Help us produce this incredible experience....,gmail.com,1358975278,...,0,1358975276,1,94 West Houston Street,US,40.727236,-74.000090,Madame X,NY,True
14325,fraudster,2013-02-05 07:30:00,996,0,US,USD,0.0,"<p><span style=""font-size: medium;""><strong>Yo...",gmail.com,1359415821,...,0,1359415820,1,3280 Peachtree Rd NE,US,33.844403,-84.370374,AQUANOX,GA,True
14332,fraudster,2013-02-24 07:00:00,667,0,US,USD,0.0,<p>Celebrates this wonderful holiday with the ...,yahoo.com,1360297993,...,0,1360297993,1,6100 North Charles Street,US,39.373780,-76.629921,Elkridge Club,MD,True


In [9]:
df.columns

Index(['acct_type', 'approx_payout_date', 'body_length', 'channels', 'country',
       'currency', 'delivery_method', 'description', 'email_domain',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'listed', 'name', 'name_length', 'num_order', 'num_payouts',
       'object_id', 'org_desc', 'org_facebook', 'org_name', 'org_twitter',
       'payee_name', 'payout_type', 'previous_payouts', 'sale_duration',
       'sale_duration2', 'show_map', 'ticket_types', 'user_age',
       'user_created', 'user_type', 'venue_address', 'venue_country',
       'venue_latitude', 'venue_longitude', 'venue_name', 'venue_state',
       'fraud'],
      dtype='object')

In [8]:
df['approx_payout_date'] = pd.to_datetime(df['approx_payout_date'], unit = 's')

In [26]:
df.org_facebook

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
14332    0.0
14333    0.0
14334    0.0
14335    0.0
14336    0.0
Name: org_facebook, Length: 14337, dtype: float64